<a href="https://colab.research.google.com/github/lorenzopaoria/Smoking-detection-and-distance-analysis/blob/main/model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.utils as vutils
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision.ops import nms
from torchvision.utils import draw_bounding_boxes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Percorso del file .pth
MODEL_PATH = '/content/drive/MyDrive/pth_epoch/model_epoch_10' # Cambia con il nome effettivo del file

# Definizione delle classi
CLASS_NAMES = ["sigarette", "smoker", "non_smoker"]

# Caricamento del dataset di test (sostituisci con il tuo dataset)
test_dataset = datasets.MNIST(root='/content/drive/MyDrive/Photo/test', train=False, download=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
def load_model(model_path):
    """Carica il modello salvato su CUDA se disponibile"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = MyModel().to(device)  # Sostituisci con la tua architettura di rete
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model, device

In [ ]:
def draw_predictions(images, boxes, labels, scores):
    """Disegna le bounding box sulle immagini"""
    images = (images * 255).byte()
    colors = ["red", "green", "blue"]
    labels_text = [f"{CLASS_NAMES[l]}: {s:.2f}" for l, s in zip(labels, scores)]
    image_with_boxes = draw_bounding_boxes(images.squeeze(0), boxes, labels=labels_text, colors=colors)
    return image_with_boxes

In [ ]:
def evaluate_model(model, device, test_loader):
    """Valuta il modello e visualizza le bounding box sulle immagini"""
    model.to(device)
    with torch.no_grad():
        for images, _ in test_loader:
            images = images.to(device)
            outputs = model(images)  # Supponiamo che restituisca boxes, labels, scores
            boxes, labels, scores = outputs["boxes"], outputs["labels"], outputs["scores"]

            # Filtra le predizioni con soglia di confidenza > 0.5
            keep = scores > 0.5
            boxes, labels, scores = boxes[keep], labels[keep], scores[keep]

            if len(boxes) > 0:
                image_with_boxes = draw_predictions(images.cpu(), boxes.cpu(), labels.cpu(), scores.cpu())
                plt.imshow(image_with_boxes.permute(1, 2, 0))
                plt.axis("off")
                plt.show()


In [ ]:
if __name__ == "__main__":
    model, device = load_model(MODEL_PATH)
    evaluate_model(model, device, test_loader)